### Step 1 - Start Spark Session and Include additional configurations and common functions

In [1]:
%run "../includes/configurations"

In [2]:
%run "../includes/common_functions"

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, FloatType

appName = "Formula 1 Database"

# Initialize a Spark session
spark = create_spark_context(appName)

24/01/04 18:35:07 WARN Utils: Your hostname, falcao-sys resolves to a loopback address: 127.0.1.1; using 192.168.11.185 instead (on interface wlx7898e8c12476)
24/01/04 18:35:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 18:35:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###  Step 2 - Create F1 Database

In [4]:
# Create the database if it doesn't exist
spark.sql("CREATE DATABASE IF NOT EXISTS f1_raw")

# Switch to the newly created database
spark.sql("USE f1_raw")

DataFrame[]

### Step 3 - Create and Save Circuits table

In [5]:
circuits_schema = StructType(fields=[
    StructField("circuitId", IntegerType(), False),
    StructField("circuitRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lng", DoubleType(), True),
    StructField("alt", IntegerType(), True),
    StructField("url", StringType(), True)
])

In [6]:
circuits_df = spark.read.option("header", True).schema(circuits_schema).csv(f"{data_folder_path}/circuits.csv")

In [7]:
circuits_df.write.saveAsTable("circuits")

### Step 4 - Create Races table and populate it

In [8]:
races_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("year", IntegerType(), True),
    StructField("round", IntegerType(), True),
    StructField("circuitId", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("date", DateType(), False),
    StructField("time", StringType(), False),
    StructField("url", StringType(), True)
])

races_df = spark.read.option("header", True).schema(races_schema).csv("../data/races.csv")

In [9]:
races_df.write.saveAsTable("races")

### Step 5 - Create Constructors table and populate it

In [10]:
constructors_schema = StructType(fields=[
    StructField("constructorId", IntegerType(), False),
    StructField("constructorRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

constructor_df = spark.read.schema(constructors_schema).json("../data/constructors.json")

In [11]:
constructor_df.write.saveAsTable("constructors")

### Step 6 - Create Drivers table and populate it

In [12]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

drives_schema = StructType(fields=[
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("name", name_schema),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

drives_df = spark.read.schema(drives_schema).json("../data/drivers.json")

In [13]:
drives_df.write.saveAsTable("drivers")

### Step 7 - Create Results table and populate it

In [14]:
results_schema = StructType(fields=[
    StructField("resultId", IntegerType(), False),
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("constructorId", IntegerType(), False),
    StructField("number", IntegerType(), True),
    StructField("grid", IntegerType(), False),
    StructField("position", IntegerType(), True),
    StructField("positionText", StringType(), False),
    StructField("positionOrder", IntegerType(), False),
    StructField("points", FloatType(), False),
    StructField("laps", IntegerType(), False),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("fastestLap", IntegerType(), True),
    StructField("rank", IntegerType(), True),
    StructField("fastestLapTime", StringType(), True),
    StructField("fastestLapSpeed", FloatType(), True),
    StructField("statusId", IntegerType(), False),
])

results_df = spark.read.schema(results_schema).json("../data/results.json")

In [15]:
results_df.write.saveAsTable("results")

### Step 8 - Create Pit Stops table and populate it

In [16]:
pit_stops_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("stop", StringType(), False),
    StructField("lap", IntegerType(), False),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), False),
    StructField("milliseconds", IntegerType(), True)
])

pit_stops_df = spark.read.schema(pit_stops_schema).option("multiLine", True).json("../data/pit_stops.json")

In [17]:
pit_stops_df.write.saveAsTable("pit_stops")

### Step 9 - Create Lap Times table and populate it

In [18]:
lap_times_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("position", IntegerType(), False),
    StructField("lap", IntegerType(), False),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

lap_times_df = spark.read.schema(lap_times_schema).csv("../data/lap_times/lap_times_split*.csv")

In [19]:
lap_times_df.write.saveAsTable("lap_times")

### Step 10 - Create Qualifying table and populate it

In [20]:
qualifying_schema = StructType(fields=[
    StructField("qualifyId", IntegerType(), False),
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("constructorId", IntegerType(), False),
    StructField("number", IntegerType(), False),
    StructField("position", IntegerType(), False),
    StructField("q1", StringType(), True),
    StructField("q2", StringType(), False),
    StructField("q3", StringType(), True)
])

qualifying_df = spark.read.schema(qualifying_schema).option("multiLine", True).json("../data/qualifying/")

In [21]:
qualifying_df.write.saveAsTable("qualifying")